In [2]:
import math 
import random
import numpy as np
import pandas as pd
import nltk
import json
import csv
import re
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, cross_validate, train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.svm import SVC
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
import tensorflow as tf
import torch 
from torch.utils.data import DataLoader, Dataset, TensorDataset, random_split, RandomSampler,IterableDataset
from torch.utils.data.sampler import SequentialSampler
from transformers import BertTokenizer
import transformers as ppb 
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import string 
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('words')
nltk.download('punkt')

stopwords.words('english')

import logging
logging.basicConfig(level=logging.ERROR)
import warnings
warnings.filterwarnings('ignore')

#needs to be GLOBAL
words = set(nltk.corpus.words.words())
stop_words = set(stopwords.words('english'))

[nltk_data] Error loading words: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1056)>
[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1056)>


In [20]:
### https://towardsdatascience.com/bert-text-classification-using-pytorch-723dfb8b6b5b
    
#!/usr/bin/env python3
import math
import numpy as np
import pandas as pd
import random
import time
import csv
import json
import datetime
import matplotlib.pyplot as plt
import re
import tensorflow as tf
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from transformers import BertModel
from torch.utils.data import DataLoader, Dataset, TensorDataset, random_split,SequentialSampler, IterableDataset
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from sklearn.model_selection import KFold, StratifiedKFold, StratifiedShuffleSplit
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer

def fetch_tweet_features(df):
    df["sentence"] = df["sentence"].str.lower()
    df['nchars'] = df['sentence'].str.len()
    df['nwords'] = df['sentence'].str.split().str.len()
    df['bhash'] = df["sentence"].str.contains(pat = '#',flags=re.IGNORECASE, regex = True).astype(int) 
    df['nhash'] = df["sentence"].str.count('#') 
    df['blink'] = df["sentence"].str.contains(pat = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', flags=re.IGNORECASE, regex = True).astype(int)
    df['nlink'] = df["sentence"].str.count(pat = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', flags=re.IGNORECASE) 
    df['bat'] = df["sentence"].str.contains(pat = '@',flags=re.IGNORECASE, regex = True).astype(int)
    df['nat'] = df["sentence"].str.count(pat = '@')
    df['brt'] =  df["sentence"].str.contains(pat = '@rt|rt@',flags=re.IGNORECASE, regex = True).astype(int) 
    ## List of  US slangs.
    slangList = ['ASAP','BBIAB','BBL','BBS','BF','BFF','BFFL','BRB','CYA','DS','FAQ','FB','FITBLR','FLBP','FML','FTFY','FTW','FYI','G2G','GF','GR8','GTFO','HBIC','HML','HRU','HTH','IDK','IGHT','IMO','IMHO','IMY','IRL','ISTG','JK','JMHO','KTHX','L8R','LMAO','LMFAO','LMK','LOL','MWF','NM','NOOB','NP','NSFW','OOAK','OFC','OMG','ORLY','OTOH','RN','ROFL','RUH','SFW','SOML','SOZ','STFU','TFTI','TIL','TMI','TTFN','TTYL','TWSS','U','W/','WB','W/O','WYD','WTH','WTF','WYM','WYSIWYG','Y','YMMV','YW','YWA']
    slangList = [x.lower() for x in slangList]
    #Checks if the sentence contains slang
    mask = df.loc[:,'sentence'].str.contains(r'\b(?:{})\b'.format('|'.join(slangList)))
    df['bslang'] = mask.astype(int)
    ## List of interjections
    interjList = ["a-ha","aaaahh","aaah","aah","adios","aha","ahem","ahh","ahhh","alas","alasbah","allhail","alleluia","aloha","argh","augh","aw","aww","awww","ay","ba-dum-tss","badum","bah","bahaha","blech","bleep","bleh","blessyou","boo","boo-hoo","booh","boohoo","brr","brrr","brrrr","bwahaha","bye","cheerio","ciao","congrats","crikey","cripes","d'oh","doh","duh","eeeek","eeek","eek","eep","eh","eureka","eww","ewww","eyh","fiddlesticks","fuff","gadzooks","gah","gesundheit","goodgrief","Good-bye","goodbye","great","grr","grrr","grrrr","ha-ha","haha","hahaha","harumph","hbahhambug","hehe","hei","heigh-ho","hey","hm","hmmm","hmmmm","holycow","holysmokes","hooray","hotdiggitydog","huh","huh","humph","hurrah","hurray","hush","hush,shush","huzzah","ich","ich","ick","ick","ick","jeepers","jeeperscreepers","jeez","la-dee-dah","la-di-da","loandbehold","man","meh","mhmm","mm","mmh","mmhm","mmm","muahaha","mwahaha","myword","nuh-uh","oh","oh","oh","ohdear","ohmy","ohwell","oh-lala","ohh","ohhoh","oi","ooh","ooh","ooh-la-la","oooh","oops","ouch","ouch","ow","ow","ow","oww","oy","oyh","pee-yew","pew","pff","pffh","pfft","phew","phooey","pow","presto","pshaw","pss","pssh","psst","right-o","sheesh","shh","shh","shoo","shoo","shoot","solong","thanks","tish","touché","tsk-tsk","ugh","ugh","ugh","uh-hu","uh-huh","uh-huh","uh-oh","uh-oh","uh-uh,","uhh","uhh","umm","umm","ummm","unh-uh","unh-unh","voila","voila","waaaaah","waah","wahoo","wee","weee","well","wham","whee","whoa","whoopee","whoops","whoosh","woohoo","wow","yahoo","yak","yay","yea","yeah","yeah","yech","yee-haw","yeeeeaah","yeehaw","yeow","yikes","yippee","yippee","yo","yoo-hoo","yoohoo","youck","youck","yow","yowza","yu-huh","yuck","yuck","yuh-hu","yuh-uh","yummy","zapello","zing"]
    interjList = [x.lower() for x in interjList]
    #Checks if the sentence contains interjection
    mask = df.loc[:, 'sentence'].str.contains(r'\b(?:{})\b'.format('|'.join(interjList)))
    df['bintj'] = mask.astype(int)
    df['tlex'] = df["sentence"].apply(lexical_diversity)
    tweet_features =  df[['tweet_id','sentence','nchars','nwords','bhash','nhash','blink','nlink','bat','nat','brt','bslang','bintj','tlex','label']]
    tweet_featuresDF = pd.DataFrame(tweet_features)
    return tweet_featuresDF

def fetch_usr_features(filename):
    usr_featuresDF = pd.DataFrame(columns=['tweet_id','sentence','usr_vrf','num_followers','num_friends','num_tweets'])
    listOfSeries = []
    with open(filename) as json_file:
        for line in json_file:
            data = json.loads(line)
            tweet_id  = int(data['id_str'])
            user_obj = data['user']
            sentence =  data['full_text'].lower()
            usr_id = user_obj['id_str']
            usr_vrf = int(user_obj['verified'])#.astype(int)
            num_followers = user_obj['followers_count']
            num_followers = math.log10(1 + num_followers)
            num_friends = user_obj['friends_count']
            num_friends = math.log10(1 + num_friends)
            num_tweets = user_obj['statuses_count']
            num_tweets = math.log10(1 + num_tweets)
            listOfSeries.append(pd.Series([int(tweet_id),sentence, usr_vrf, num_followers, num_friends, num_tweets ], index=usr_featuresDF.columns ))
        usr_featuresDF = usr_featuresDF.append(listOfSeries , ignore_index=True)
    return usr_featuresDF

def lexical_diversity(text):
    return len(set(text.split())) / len(text.split())

class CustomCombinedDataset(Dataset):
    def __init__(self,name):
        #self.tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-cased')
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
        #self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
        if name == 'crisismmd':
            self.df = pd.DataFrame()
            files = ["/home/renato/Downloads/CrisisMMD_v2.0/crisismmd_datasplit_all/task_informative_text_img_train.tsv","/home/renato/Downloads/CrisisMMD_v2.0/crisismmd_datasplit_all/task_informative_text_img_dev.tsv","/home/renato/Downloads/CrisisMMD_v2.0/crisismmd_datasplit_all/task_informative_text_img_test.tsv"]
            #files = ["/home/joao/task_informative_text_img_train.tsv","/home/joao/task_informative_text_img_dev.tsv","/home/joao/task_informative_text_img_test.tsv"]
            for filename in files:
                tmpdf = pd.read_csv(filename,delimiter='\t',quoting=csv.QUOTE_NONE,error_bad_lines=False,encoding='utf-8')
                tmpdf = tmpdf[['tweet_id','tweet_text','label_text']]
                tmpdf = tmpdf.rename(columns={'tweet_text': 'sentence'})
                tmpdf = tmpdf.rename(columns={'label_text': 'label'})
                tmpdf['label'] = tmpdf['label'].replace('informative', 1)
                tmpdf['label'] = tmpdf['label'].replace('not_informative', 0)
                self.df = self.df.append(tmpdf,ignore_index = True)
            self.df = self.df.drop_duplicates(subset=['tweet_id','sentence','label'])
            self.df = self.df.reset_index(drop=True)
            twt_feat = fetch_tweet_features(self.df)
            twt_feat.to_csv("./crisismmd.ORG.tsv", sep="\t")
            usr_features = fetch_usr_features("/home/renato/Downloads/CrisisMMD_v2.0/crisismmd_datasplit_all/crisismmd.json")
            #usr_features = fetch_usr_features("/home/joao/crisismmd.json")
            series = []
            for index, row in usr_features.iterrows():
                tweet_id = row['tweet_id']
                usr_vrf, num_followers, num_friends, num_tweets = row['usr_vrf'],row['num_followers'],row['num_friends'],row['num_tweets']
                idx = twt_feat[twt_feat['tweet_id']==tweet_id].index.values.astype(int)[0]
                tweet_features  = twt_feat.loc[idx]
                series.append(pd.Series([tweet_features['tweet_id'], tweet_features['sentence'],tweet_features['nchars'],tweet_features['nwords'],tweet_features['bhash'],tweet_features['nhash'],tweet_features['blink'],tweet_features['nlink'],tweet_features['bat'],tweet_features['nat'],tweet_features['brt'],tweet_features['bslang'], tweet_features['bintj'], tweet_features['tlex'], usr_vrf, num_followers, num_friends, num_tweets, tweet_features['label']],index=list(['tweet_id','sentence','nchars','nwords','bhash','nhash','blink','nlink','bat','nat','brt','bslang','bintj','tlex','usr_vrf','num_followers', 'num_friends', 'num_tweets','label'])))
            self.df = pd.DataFrame(series)
            self.df = self.df.drop_duplicates(subset=['tweet_id','sentence','nchars','nwords','bhash','nhash','blink','nlink','bat','nat','brt','bslang','bintj','tlex','usr_vrf', 'num_followers', 'num_friends', 'num_tweets','label'])
            self.df =  self.df.reset_index(drop=True) 
            print((self.df['label'].value_counts()))
            self.df.to_csv("./crisismmd.subset.tsv", sep="\t")
        if name == 'covid':
            self.df =  pd.DataFrame()
            #"/home/renato/Datasets/COVID19Tweet-master/train.tsv",
            #files = ["/home/joao/COVID19Tweet-master/train.tsv","/home/joao/COVID19Tweet-master/valid.tsv"]
            files = ["/home/renato/Datasets/COVID19Tweet-master/train.tsv","/home/renato/Datasets/COVID19Tweet-master/valid.tsv"]
            for filename in files:
                #tmpdf = pd.read_csv(filename,delimiter='\t',encoding='utf-8') 
                tmpdf = pd.read_csv(filename,delimiter='\t',quoting=csv.QUOTE_NONE,error_bad_lines=False,encoding='utf-8',lineterminator="\n")
                tmpdf = tmpdf.rename(columns={'Id': 'tweet_id'})        
                tmpdf = tmpdf.rename(columns={'Text': 'sentence'})
                tmpdf = tmpdf.rename(columns={'Label': 'label'})
                tmpdf['label'] = tmpdf['label'].replace('INFORMATIVE', 1)
                tmpdf['label'] = tmpdf['label'].replace('UNINFORMATIVE', 0)
                self.df = self.df.append(tmpdf,ignore_index = True)
            self.df = self.df.drop_duplicates(subset=['tweet_id','sentence','label'])
            self.df = self.df.reset_index(drop=True)
            twt_feat  =  fetch_tweet_features(self.df)
            twt_feat.to_csv("./covid.ORG.tsv", sep="\t")
            usr_features = fetch_usr_features("/home/renato/Datasets/COVID19Tweet-master/covid.json")
            #usr_features = fetch_usr_features("/home/joao/covid.json")
            series = []
            for index, row in usr_features.iterrows():
                tweet_id = int(row['tweet_id'])
                usr_vrf, num_followers, num_friends, num_tweets = row['usr_vrf'],row['num_followers'],row['num_friends'],row['num_tweets']
                #print(tweet_id)
                idx = twt_feat[twt_feat['tweet_id']==tweet_id].index.values.astype(int)[0]
                tweet_features  = twt_feat.loc[idx]
                series.append(pd.Series([tweet_features['tweet_id'], tweet_features['sentence'],tweet_features['nchars'],tweet_features['nwords'],tweet_features['bhash'],tweet_features['nhash'],tweet_features['blink'],tweet_features['nlink'],tweet_features['bat'],tweet_features['nat'],tweet_features['brt'],tweet_features['bslang'], tweet_features['bintj'], tweet_features['tlex'], usr_vrf, num_followers, num_friends, num_tweets, tweet_features['label']],index=list(['tweet_id','sentence','nchars','nwords','bhash','nhash','blink','nlink','bat','nat','brt','bslang','bintj','tlex','usr_vrf','num_followers', 'num_friends', 'num_tweets','label'])))
            self.df = pd.DataFrame(series)
            self.df = self.df.drop_duplicates(subset=['tweet_id','sentence','nchars','nwords','bhash','nhash','blink','nlink','bat','nat','brt','bslang','bintj','tlex','usr_vrf', 'num_followers', 'num_friends', 'num_tweets','label'])
            self.df =  self.df.reset_index(drop=True) 
            print((self.df['label'].value_counts()))
            self.df.to_csv("./covid.subset.tsv", sep="\t")
        if name == 'crisislext26':
            self.df = pd.read_csv("/home/renato/Datasets/CrisisLexT26/crisisLexT26.csv", encoding='utf-8')
            #self.df = pd.read_csv("/home/joao/crisisLexT26.csv", encoding='utf-8')
            self.df = self.df.drop([' Information Source', ' Information Type' ], axis=1)
            self.df = self.df.rename(columns={'Tweet ID': 'tweet_id'})
            self.df = self.df.rename(columns={' Tweet Text': 'sentence'})
            self.df = self.df.rename(columns={' Informativeness': 'label'})
            self.df = self.df.reset_index(drop=True) 
            twt_feat  = fetch_tweet_features(self.df)
            d2 = twt_feat 
            d2 = d2[d2.label!= 'Not related']
            d2 = d2[d2.label!= 'Not applicable']
            d2['label'] = d2['label'].replace('Related and informative', 1)
            d2['label'] = d2['label'].replace('Related - but not informative', 0)
            d2.to_csv("./crisislext26.ORG.tsv", sep="\t")
            #usr_features = fetch_usr_features("/home/joao/crisisLexT26.json")
            usr_features = fetch_usr_features("/home/renato/Datasets/CrisisLexT26/crisisLexT26.json")
            series = []
            for index, row in usr_features.iterrows():
                tweet_id = int(row['tweet_id'])
                usr_vrf, num_followers, num_friends, num_tweets = row['usr_vrf'],row['num_followers'],row['num_friends'],row['num_tweets']
                idx = twt_feat[twt_feat['tweet_id'].astype(int)==int(tweet_id)].index.values.astype(int)[0]
                tweet_features  = twt_feat.loc[idx]
                series.append(pd.Series([tweet_features['tweet_id'], tweet_features['sentence'],tweet_features['nchars'],tweet_features['nwords'],tweet_features['bhash'],tweet_features['nhash'],tweet_features['blink'],tweet_features['nlink'],tweet_features['bat'],tweet_features['nat'],tweet_features['brt'],tweet_features['bslang'], tweet_features['bintj'], tweet_features['tlex'], usr_vrf, num_followers, num_friends, num_tweets, tweet_features['label']],index=list(['tweet_id','sentence','nchars','nwords','bhash','nhash','blink','nlink','bat','nat','brt','bslang','bintj','tlex','usr_vrf','num_followers', 'num_friends', 'num_tweets','label'])))
            self.df = pd.DataFrame(series)
            self.df =  self.df.reset_index(drop=True) 
            self.df = self.df[self.df.label!= 'Not related']
            self.df = self.df[self.df.label!= 'Not applicable']
            self.df['label'] = self.df['label'].replace('Related and informative', 1)
            self.df['label'] = self.df['label'].replace('Related - but not informative', 0)
            self.df = self.df.drop_duplicates(subset=['tweet_id','sentence','nchars','nwords','bhash','nhash','blink','nlink','bat','nat','brt','bslang','bintj','tlex','usr_vrf', 'num_followers', 'num_friends', 'num_tweets','label'])
            self.df = self.df.reset_index(drop=True)
            self.df.to_csv("./crisislext26.subset.tsv", sep="\t")
        if name == 'crisislext6':
            self.df = pd.read_csv("/home/renato/Datasets/CrisisLexT6-v1.0/CrisisLexT6/crisisLexT6.csv",encoding='utf-8') 
            #self.df = pd.read_csv("/home/joao/crisisLexT6.csv", encoding='utf-8')
            self.df = self.df.rename(columns={' tweet': 'sentence'})
            self.df = self.df.rename(columns={' label': 'label'})
            self.df = self.df.rename(columns={'tweet id': 'tweet_id'})
            self.df['label'] = self.df['label'].replace('on-topic', 1)
            self.df['label'] = self.df['label'].replace('off-topic', 0)
            self.df = self.df.drop_duplicates(subset=['tweet_id','sentence','label'])
            self.df['tweet_id'] = self.df['tweet_id'].apply(lambda x: x.replace("'", ''))
            self.df = self.df.drop_duplicates(subset=['tweet_id','sentence','label'])
            self.df = self.df.reset_index(drop=True) 
            twt_feat  = fetch_tweet_features(self.df)
            twt_feat.to_csv("./crisislext6.ORG.tsv", sep="\t")
            usr_features = fetch_usr_features("/home/renato/Datasets/CrisisLexT6-v1.0/CrisisLexT6/crisisLexT6.json")
            #usr_features = fetch_usr_features("/home/joao/crisisLexT6.json")
            series = []
            for index, row in usr_features.iterrows():
                tweet_id = int(row['tweet_id'])
                usr_vrf, num_followers, num_friends, num_tweets = row['usr_vrf'],row['num_followers'],row['num_friends'],row['num_tweets']
                idx = twt_feat[twt_feat['tweet_id'].astype(int)==int(tweet_id)].index.values.astype(int)[0]
                tweet_features  = twt_feat.loc[idx]
                series.append(pd.Series([tweet_features['tweet_id'], tweet_features['sentence'],tweet_features['nchars'],tweet_features['nwords'],tweet_features['bhash'],tweet_features['nhash'],tweet_features['blink'],tweet_features['nlink'],tweet_features['bat'],tweet_features['nat'],tweet_features['brt'],tweet_features['bslang'], tweet_features['bintj'], tweet_features['tlex'], usr_vrf, num_followers, num_friends, num_tweets, tweet_features['label']],index=list(['tweet_id','sentence','nchars','nwords','bhash','nhash','blink','nlink','bat','nat','brt','bslang','bintj','tlex','usr_vrf','num_followers', 'num_friends', 'num_tweets','label'])))
            self.df = pd.DataFrame(series)
            self.df = self.df.drop_duplicates(subset=['tweet_id','sentence','nchars','nwords','bhash','nhash','blink','nlink','bat','nat','brt','bslang','bintj','tlex','usr_vrf', 'num_followers', 'num_friends', 'num_tweets','label'])
            self.df =  self.df.reset_index(drop=True) 
            self.df.to_csv("./crisislext6.subset.tsv", sep="\t")
        self.df["sentence"] = self.df["sentence"].str.lower()
        #################
    def lexical_diversity(self,text):
        return len(set(text.split())) / len(text.split())
 
def main():

    datasets = ['covid', 'crisislext6', 'crisislext26', 'crisismmd']
    for data in datasets :
        print("=== {} ===".format(data))
        if data == 'crisismmd':
            dataset = CustomCombinedDataset("crisismmd")
        if data == 'covid':
            dataset =  CustomCombinedDataset("covid")
        if data ==  'crisislext6':
            dataset =  CustomCombinedDataset("crisislext6")
            # Create a 90-10 train-validation split.
            #train_size = int(0.9 * len(dataset))
            #val_size = len(dataset) - train_size
            #train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
        if data == 'crisislext26':
            dataset =  CustomCombinedDataset("crisislext26")
            # Create a 90-10 train-validation split.
            #train_size = int(0.9 * len(dataset))
            #val_size = len(dataset) - train_size
            #train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

main()
        

=== covid ===


I1004 19:54:08.772951 140612314093376 tokenization_utils_base.py:1254] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/renato/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


0    3816
1    3378
Name: label, dtype: int64
=== crisislext6 ===


I1004 19:54:25.028251 140612314093376 tokenization_utils_base.py:1254] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/renato/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


=== crisislext26 ===


I1004 20:01:48.028802 140612314093376 tokenization_utils_base.py:1254] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/renato/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


=== crisismmd ===


I1004 20:02:37.174732 140612314093376 tokenization_utils_base.py:1254] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/renato/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


1    9343
0    3443
Name: label, dtype: int64
